# Select, rename and define a re-reference method for .edf files of an EEG database 
This notebook allows you to select channels of interest within your .edf EEG database, rename them, and define a re-reference method. \
It returns a JSON file (corresponding to a python dictionary) that you can use later in your analysis to apply the choice you made here. \
It is especially useful for datasets with different eeg montage (e.g. multicentric dataset). 

---
**To use this notebook, read each text box (called markdown cells in Jupyter notebook), run its associated code box (code cell), interact with the widgets (buttons and text box), and read the output below.**\
To run a cell, select the cell and either click on "Run" and "Run Selected Cell" in the menu bar, or press "Shift+Enter". \
\
This notebook is organized into 4 sections (with a section 0 to prepare your notebook):
1. Select your study folder, extract the data's information, and dispaly channels configurations
2. Select channels of interest
3. Rename channels
4. Define re-reference method
5. Preview and save JSON file
6. Test the JSON file

Once you are done with one section, you can reduce it by pressing the down arrow on the left of the section title, to improve the readability.\
\
_The notebook will save a .json file in a folder config_param within your study folder._ \
_To load this file as a python dictionary:\
define a path variable_ ``json_path = "your_path"``\
_and use_ ``with open(json_path, "r", encoding="utf-8") as f: config_dict = json.load(f)``

---
This notebook was developed on the APOMORPHEE database and tested on ICEBERG.  \
last update 20/09/2025, YN

<hr style="height:4px; background-color:black; border:none;">

## 0. Import packages and define custom functions

The code cell below loads the packages required for this notebook and defines custom functions that we will use.\
If you have already installed all the packages, you will get the message ``✅Packages and functions successfully imported!``.

In [ ]:
# Import cell
try:
    import os
    import re
    import mne
    import math
    import json
    import chardet
    import warnings
    import traceback
    import numpy as np
    import pandas as pd
    from pathlib import Path
    import ipywidgets as widgets
    from textwrap import shorten
    from collections import OrderedDict
    from ipyfilechooser import FileChooser
    from IPython.display import display, HTML, clear_output
except ImportError as e:
    print("⚠️ Error: ", e)
else:
    print("✅ Packages and functions successfully imported!")

# custom function to detect automatically and return the encoding of edf file
def detect_encoding(byte_string, min_confidence=0.6):
    result = chardet.detect(byte_string)
    encoding = result['encoding']
    confidence = result['confidence']
    if encoding is None or confidence < min_confidence:
        raise UnicodeDecodeError("chardet", byte_string, 0, len(byte_string),
                                 f"\tUnable to reliably detect encoding. Detected: {encoding} with confidence {confidence}")
    return encoding

# custom function to read information from EDF headers, without using the pyedflib package (that was too strict for ICEBERG)
# EDF file should follow a strict format, dedicating a specific number of octets for each type of information.
# it means that we can read the info octet by octet by specifying the number of octets we expect for the next variable (that is known from the EDF norm)
def read_edf_header_custom(file_path):
    with open(file_path, 'rb') as f: # open the file in binary mode, to read octet by octet. 
        header = {}
        # detect encoding
        raw_header = f.read(256)
        encoding = detect_encoding(raw_header)
        # print(f"\tDetected encoding for {file_path} : {encoding}")
        # Rewind to the beginning of the file
        f.seek(0)
        
        # the first 256 octets are global subject info
        header['version'] = f.read(8).decode(encoding).strip()
        header['patient_id'] = f.read(80).decode(encoding).strip()
        header['recording_id'] = f.read(80).decode(encoding).strip()
        header['start_date'] = f.read(8).decode(encoding).strip()
        header['start_time'] = f.read(8).decode(encoding).strip()
        header['header_bytes'] = int(f.read(8).decode(encoding).strip())
        header['reserved'] = f.read(44).decode(encoding).strip()
        header['n_data_records'] = int(f.read(8).decode(encoding).strip())
        header['duration_data_record'] = float(f.read(8).decode(encoding).strip())
        header['n_channels'] = int(f.read(4).decode(encoding).strip())
        
        # get info per channel
        n = header['n_channels']
        channel_fields = {
            'channel': [],
            'transducer_type': [],
            'dimension': [],
            'physical_min': [],
            'physical_max': [],
            'digital_min': [],
            'digital_max': [],
            'prefiltering': [],
            'sampling_frequency': [],
            'reserved': [],
        }

        for key in channel_fields:
            length = {
                'channel': 16,
                'transducer_type': 80,
                'dimension': 8,
                'physical_min': 8,
                'physical_max': 8,
                'digital_min': 8,
                'digital_max': 8,
                'prefiltering': 80,
                'sampling_frequency': 8,
                'reserved': 32,
            }[key]
            channel_fields[key] = [f.read(length).decode(encoding).strip() for _ in range(n)]

        header.update(channel_fields)
    
    return header

# function to extract filter information from the string in headers
def extract_filter_value(s, tag):
    if pd.isna(s):
        return None
    match = re.search(rf'{tag}[:\s]*([\d\.]+)\s*', s, re.IGNORECASE)
    return float(match.group(1)) if match else None

# custom function to get the sampling frequency out of a dataframe (the df needs to have 'subject' and 'channel' as columns)
def get_sf(df, subject, channel):
    df_sf = df[(df['subject'] == subject) & (df['channel'] == channel)]
    if not df_sf.empty:
        return df_sf.iloc[0]['sampling_frequency']
    else:
        return None

# function to create a widget slider to select the configuration to inspect
def mk_config_slider(value = 1, min = 1, max = 5):
    config_slider = widgets.IntSlider(
    value=value,
    min=min,
    max=max,
    step=1,
    description='Selected configuration:',
    style={'description_width': '150px'},   # increase description width (to adjust based on the description)
    layout=widgets.Layout(width='400px'),   # to adjust widget size
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
    return config_slider

# function to print the configuration of a dataset parameter
def print_config(i, config_dict, param):
    # get the key and value from the dictionary
    idx = i - 1
    # get participant ID
    value = list(config_dict.values())  
    v = value[idx]  
    # get configuration
    key = list(config_dict.keys())
    k = key[idx]
    
    # print info
    print(f'Selected configuration: # {i}')
    print(f'\t{len(k)} {param}: {k}')
    print(f'\t{len(v)} participants: {v}')

# function to create a scrollable box for long output (e.g., cell loading the data) 
def print_in_scrollable_box(text, height=300, font_size="12px"):
    display(HTML(f'<pre style="overflow-y:scroll; height:{height}px; border:1px solid black; padding:10px; font-size:{font_size};">{text}</pre>'))

# STATE is note really used in this notebook, but added to debug the voila version
STATE = SimpleNamespace(
    folder_path=None,
    config_param_path=None,
    edf_files=[],
    configs=None,
    config_labels=None,
    selected_by_config_raw=None,
    selected_by_config_canonical_prelim=None,
    selected_by_config_canonical=None,
    ch_config_dict=None,
    remap_by_config=None,
    reref_plan_by_config=None,
    per_subject_dict=None,
    COMMON_10_10=None,
    json_path=None,
)


<hr style="height:4px; background-color:black; border:none;">

## 1. Select study folder and extract EEGs/EOGs information

### 1.1 Select the study folder

The code cell below will open a widget to select the folder containing your data.

In [ ]:
# call widget to select your data folder 
chooser = FileChooser(os.getcwd())
chooser.title = "<b>Choose your study folder</b>"
chooser.show_only_dirs = True

# Define output widget to redirect the print within the function
out = widgets.Output()

# custom function to extract the folder path once the folder has been chosen: 
def on_folder_selected(chooser):
    out.clear_output()
    with out:
        chooser.folder_path = chooser.selected_path
        print("📁 Selected Path:", chooser.folder_path)
        
        # get the edf file list 
        chooser.edf_files = [
            f for f in Path(chooser.folder_path).rglob('*.edf')
            if not f.name.startswith('._') # don't select files starting with ._ (that can be found in mac for example)
            ]
        if not chooser.edf_files:
            print(f"⚠️ There is no .edf file in your folder")
        else:
            print(f"\nThere is {len(chooser.edf_files)} .edf files in your folder!")
        
        # check the existence and/or create the config_param folder that will receive the outputs from this notebook to relabel and re-ref your data
        chooser.config_param_path = f'{chooser.folder_path}/config_param'
        if not os.path.exists(chooser.config_param_path):
            os.makedirs(chooser.config_param_path)
            print("\nCreated config. parameter folder at: " + chooser.config_param_path)
        else:
            print("\nConfig. parameter folder already exists. \nPrevious parameters (if any) will be overwritten at: \n" + chooser.config_param_path)

# callback to run the function only when a folder is selected
chooser.register_callback(on_folder_selected)
display(chooser, out)

### 1.2 Extract infromation from each file parameters for each participant

The code cell below will loop across the .edf files to extract the information of EEGs and EOGs from each subject.\
It returns a table that can easily be manipulated to access specific information in the rest of the notebook.\
If some files failed to load, their names will be saved in a .tsv file in the summary folder.

In [ ]:
# get variables from the chooser widget
folder_path = chooser.folder_path
config_param_path = chooser.config_param_path
edf_files = chooser.edf_files
STATE.edf_files = edf_files

# 
table_found = False
found_group = False

# Initialize a list of dataframes to store file info, which will be concatenated at the end (this is better for performance)
df_list = []
# Initialize an empty list for files that could not be read
failed_list = []

# intialize a dynamic output
output = ""
dynamic_out = widgets.Output()
display(dynamic_out)

for e, edf_path in enumerate(edf_files):
    with dynamic_out:
        output += (f'file {e+1}/{len(edf_files)}, currently opening file: {edf_path}\n')
        dynamic_out.clear_output(wait=True)
        print_in_scrollable_box(output, font_size = "12px")
        
        # read file with the custom function
        try:
            edf_header = read_edf_header_custom(edf_path) 
            
            # get subject name (corresponding to file_name)
            sub_name = edf_path.stem
            
            # get subject group (from the parent folder because in the ICEBERG database subfolders were created per patient group)
            sub_folder = edf_path.parent.name # get the parent folder of the subject file (path)
            
            # create df from signal info
            df = pd.DataFrame(edf_header)
                
            # theoretical resolution (edf are 16bit files so the eeg signal can take 2^16 values within the dynamic range)
            df['res_theoretical'] = (abs(pd.to_numeric(df['physical_min']))+abs(pd.to_numeric(df['physical_max'])))/pow(2,16)
            # turn theoretical resolution to uV if dimension is mV (if no dimension, it is a mess)
            df.loc[df['dimension'].str.contains('mv', case=False, na=False), 'res_theoretical'] *= 1000
            
            # get filtering info in different columns
            df['lowpass']   = df['prefiltering'].apply(lambda x: extract_filter_value(x, 'LP'))
            df['highpass']  = df['prefiltering'].apply(lambda x: extract_filter_value(x, 'HP'))
            df['notch']  = df['prefiltering'].apply(lambda x: extract_filter_value(x, 'NOTCH'))
            
            # add subject info in the dataframe
            df['subject'] = sub_name
            df['sub_folder'] = sub_folder
            df['group'] = np.nan # initialyze column 'group' with NaN
            # get group from participants table if any (else group will be inferred from subfolder or filename extension later)
            if found_group:
                df['group'] = subj_table.loc[subj_table['participant_id'] == sub_name, 'group'].iloc[0]

            # extract filename component before and after subject number (so we assume subject name contains at least incrementing numbers that are at the beginning of the file name)  
            #   ^       → start of string  
            # (.*?)     → group 1: as few chars as possible, up to the first digit  
            # (\d+)     → group 2: the number itself  
            # (.*)      → group 3: the rest of the string  
            # $         → end of string
            pre_comp = sub_num = post_comp = np.nan
            pattern = re.compile(r'^(.*?)(\d+)(.*)$')
            m = pattern.match(sub_name)
            if m:
                pre_comp = m.group(1) or np.nan
                sub_num = m.group(2) or np.nan
                post_comp = m.group(3) or np.nan
            df['pre_fn_comp'] = pre_comp
            df['post_fn_comp'] = post_comp
            df['sub_num'] = sub_num
            
            df['path'] = str(edf_path)
            df['session'] = np.nan # session will be inferred later from file name component
            
            # select only the columns of interest
            df = df[['subject', 'group', 'session', 'path', 'sub_folder', 'sub_num', 'pre_fn_comp', 'post_fn_comp', 'channel', 'transducer_type', 'dimension', 'sampling_frequency', 
                 'highpass', 'lowpass', 'notch', 'physical_min', 'physical_max', 'res_theoretical']]
            
            # store subject data
            df_list.append(df)
    
        except UnicodeDecodeError as e:
            err = f"⚠️ Encoding problem for {edf_path}\n"
            output += err
            clear_output(wait=True)
            print_in_scrollable_box(output, font_size="12px")
            failed_list.append((edf_path, 'encoding'))
        except Exception as e:
            # tb = traceback.format_exc()
            err = f"❌ Unexpected problem for {edf_path} : {e}\n"
            output += err
            clear_output(wait=True)
            print_in_scrollable_box(output, font_size="12px")
            failed_list.append((edf_path, 'other'))
   
# concatenate dataframe into one and only
with warnings.catch_warnings(): # this is to skip a warning not affecting our operation
    warnings.simplefilter("ignore", FutureWarning)
    df_full = pd.concat(df_list, ignore_index=True)

# save the failed list if not empty:
failed_df = pd.DataFrame(failed_list)
if not failed_df.empty:
    failed_df.to_csv(f'{config_param_path}/failed_edf_read.tsv', sep = '\t')
    print(f'\nSaving the list of files that could not be read to: \n{config_param_path}/failed_edf_read.tsv')

# select only EEG and EOGs channels and return a warning if the number of participant is smaller/higher
# define common EEG label from the 10-10 convention
COMMON_EEG_label = r'\bFp1\b|\bFpz\b|\bFp2\b|\bAF7\b|\bAF3\b|\bAFz\b|\bAF4\b|\bAF8\b|\bF7\b|\bF5\b|\bF3\b|\bF1\b|\bFz\b|\bF2\b|\bF4\b|\bF6\b|\bF8\b|\bFT7\b|\bFC5\b|\bFC3\b|\bFC1\b|\bFCz\b|\bFC2\b|\bFC4\b|\bFC6\b|\bFT8\b|\bT7\b|\bC5\b|\bC3\b|\bC1\b|\bCz\b|\bC2\b|\bC4\b|\bC6\b|\bT8\b|\bTP7\b|\bCP5\b|\bCP3\b|\bCP1\b|\bCPz\b|\bCP2\b|\bCP4\b|\bCP6\b|\bTP8\b|\bP7\b|\bP5\b|\bP3\b|\bP1\b|\bPz\b|\bP2\b|\bP4\b|\bP6\b|\bP8\b|\bPO7\b|\bPO5\b|\bPO3\b|\bPOz\b|\bPO4\b|\bPO6\b|\bPO8\b|\bO1\b|\bOz\b|\bO2\b|\bM1\b|\bM2\b|EEG'
mask = df_full['transducer_type'].str.contains(r'\bEEG\b|\bAGAGCL ELECTRODE\b|\bEOG\b', case=False, na=False) | df_full['channel'].str.contains(r'EOG', case=False, na=False) | df_full['channel'].str.contains(COMMON_EEG_label, case = False, na=False)
df_ch = df_full[mask]
# remove the emg/ecg channels that were captured with the AGAGCL ELECTRODE transducer type 
df_ch = df_ch[~df_ch['channel'].str.contains(r'emg|ecg', case=False, na=False)] # the ~ allows to not select the selection (like ! in matlab)

# get the EEG configuration per participant 
ch_per_sub = df_ch.groupby('subject')['channel'].apply(lambda x: tuple(sorted(set(x))))

# identify the channel configuration of each participant and store them in a dict to print per channel config
ch_config_dict = {}
for config in ch_per_sub.unique():
    sub = ch_per_sub[ch_per_sub == config].index.tolist()
    ch_config_dict[config] = sub

if len(ch_config_dict) > 1:
    print('\n>>> There is multiple EEG configurations in your dataset! <<<')    
    print(f'\n\tNumber of different configuration: {len(ch_config_dict)}\n')
else:
    print('\n>>> There is only one EEG configuration in your dataset! <<<\n')


### 1.3 Display channels configurations

The code cell below will display the different existing channels configuration within your dataset.\
You should inspect it to identify the channels shared across your configurations to guide your selection of channels.   

In [ ]:
# ------- A. Construire un tableau "aligné" colonnes=config -------
# ch_config_dict: { tuple(sorted(set(channels))) : [list_of_subjects] }
configs = list(ch_config_dict.keys())
n_configs = len(configs)

# prépares des étiquettes lisibles "Cfg 1 (n=12)  [ex: sub1, sub2, ...]"
col_labels = []
for i, cfg in enumerate(configs, start=1):
    subs = ch_config_dict[cfg]
    n = len(subs)
    # petit aperçu des participants dans l'en-tête (tronqué)
    preview = shorten(", ".join(subs[:5]), width=40, placeholder="…")
    col_labels.append(f"config. {i}<br>(n={n})")

# liste triée des canaux par config
cfg_channel_lists = [sorted(list(cfg)) for cfg in configs]
max_len = max(len(lst) for lst in cfg_channel_lists) if cfg_channel_lists else 0

# on padde les colonnes à la même hauteur
data = {}
for label, ch_list in zip(col_labels, cfg_channel_lists):
    padded = ch_list + [""] * (max_len - len(ch_list))
    data[label] = padded

df_configs_aligned = pd.DataFrame(data)
df_configs_aligned.index = pd.Index(range(1, max_len+1), name="rank")

display(HTML(df_configs_aligned.to_html(escape=False)))

<hr style="height:4px; background-color:black; border:none;">

## 2. Select channels of interest

The code cell below will open a widget where you can select your channels of interest per configuration.
- Click a configuration to unfold its channels list.
- Click on the checkboxes to un/select a channel.
- You can use the textbox to search for specific channels, and the buttons to un/select all channels.
- Repeat this operation for each configuration.
- Click the "Save selection" button to save your selection, a short summary will be displayed.\
Make sure to click on the "Save selection" button before moving on to the next step.

In [ ]:
def normalize_label(x):
    return "" if x is None else str(x).strip()

# --- Prépare les données depuis ch_config_dict/configs ---
# configs : liste de tuples/channels (déjà construite dans ton code précédent)
# ch_config_dict : { tuple(sorted(set(channels))) : [list_of_subjects] }

if 'configs' not in globals():
    configs = list(ch_config_dict.keys())

config_labels = []
channels_by_cfg = []   # liste parallèle aux labels : list[str] de canaux raw par config

for i, cfg in enumerate(configs, start=1):
    subs = ch_config_dict[cfg]
    n = len(subs)
    label = f"config. {i} (n={n})"
    config_labels.append(label)
    channels_by_cfg.append(sorted(list(cfg)))

# --- Calcul canaux communs (optionnel : bouton "communs" utile) ---
canonical_by_cfg = [sorted({normalize_label(ch) for ch in cfg if normalize_label(ch)}) for cfg in channels_by_cfg]
common_canonical = set(canonical_by_cfg[0]) if canonical_by_cfg else set()
for cset in canonical_by_cfg[1:]:
    common_canonical &= set(cset)

# --------- Fabrique un panneau par configuration (avec recherche) ---------
def build_config_panel(ch_list):
    """
    Retourne (container_widget, state_dict) pour une configuration donnée.
    state_dict contient: 'checkboxes', 'filter', 'count_label'
    """
    # Widgets de contrôle
    filter_box   = widgets.Text(placeholder="Filter channels (regex or text)…", layout=widgets.Layout(width="320px"))
    btn_all      = widgets.Button(description="select all", tooltip="Sélectionner tous les canaux")
    btn_none     = widgets.Button(description="deselect all", tooltip="Déselectionner tous les canaux")
    btn_invert   = widgets.Button(description="inverse selection", tooltip="Inverser la sélection")
    # btn_common   = widgets.Button(description="Commun(s)", tooltip="Garder uniquement les canaux communs (canonisés)")

    # Cases à cocher (une par canal)
    checkboxes = [widgets.Checkbox(value=True, description=ch) for ch in ch_list]  # par défaut: tout coché
    count_label = widgets.HTML()  # affichera "X / N sélectionnés"

    # Mise à jour du compteur
    def update_count():
        sel = sum(cb.value for cb in checkboxes)
        total = len(checkboxes)
        count_label.value = f"<b>{sel}</b> / {total} sélectionnés"

    update_count()

    # Handlers boutons
    def on_all(_):
        for cb in checkboxes:
            cb.value = True
        update_count()

    def on_none(_):
        for cb in checkboxes:
            cb.value = False
        update_count()

    def on_invert(_):
        for cb in checkboxes:
            cb.value = not cb.value
        update_count()

    # def on_common(_):
    #     # On garde cochés seulement les canaux dont la forme canonisée est dans l'intersection
    #     keep = {cb: (normalize_label(cb.description) in common_canonical) for cb in checkboxes}
    #     for cb, k in keep.items():
    #         cb.value = bool(k)
    #     update_count()

    btn_all.on_click(on_all)
    btn_none.on_click(on_none)
    btn_invert.on_click(on_invert)
    # btn_common.on_click(on_common)

    # Filtrage (afficher/masquer visuellement selon filtre)
    # Accepte une regex ; si regex invalide, on tombe en "contains" insensible à la casse
    out_box = widgets.VBox(checkboxes, layout=widgets.Layout(max_height="300px", overflow="auto", border="1px solid #ddd", padding="4px"))

    def apply_filter(*args):
        patt = filter_box.value.strip()
        for cb in checkboxes:
            show = True
            label = cb.description
            if patt:
                try:
                    show = bool(re.search(patt, label, flags=re.IGNORECASE))
                except re.error:
                    show = patt.lower() in label.lower()
            cb.layout.display = "" if show else "none"

    filter_box.observe(apply_filter, names="value")

    # Chaque checkbox met à jour le compteur
    for cb in checkboxes:
        cb.observe(lambda ch: update_count(), names="value")

    controls = widgets.HBox([filter_box, btn_all, btn_none, btn_invert], layout=widgets.Layout(gap="8px", flex_flow="row wrap"))
    footer   = widgets.HBox([count_label])

    panel = widgets.VBox([controls, out_box, footer])
    state = {"checkboxes": checkboxes, "filter": filter_box, "count_label": count_label}
    return panel, state

# --------- Construire l’Accordion global ---------
panels = []
states = []  # un state par config
for ch_list in channels_by_cfg:
    panel, st = build_config_panel(ch_list)
    panels.append(panel)
    states.append(st)

acc = widgets.Accordion(children=panels)
for i, lbl in enumerate(config_labels):
    acc.set_title(i, lbl)

display(acc)

# --------- Bouton pour récupérer la sélection dans deux variables ---------
btn_save = widgets.Button(description="Save selection", button_style="success", icon="save")
save_out = widgets.Output()

def collect_selection(_=None):
    """
    Construit deux dicts:
      - selected_by_config_raw:  {config_label: [canaux 'raw' cochés]}
      - selected_by_config_canon: {config_label: [canaux canonisés (uniques)]}
    Les deux variables sont créées/écrasées dans l'espace global du notebook.
    """
    selected_raw = {}
    selected_canon = {}

    for lbl, st, ch_list in zip(config_labels, states, channels_by_cfg):
        # réassocier proprement description -> checkbox
        # (l’ordre de ch_list correspond à l’ordre de création)
        checked = []
        for cb in st["checkboxes"]:
            if cb.value:
                checked.append(cb.description)

        selected_raw[lbl] = checked
        # version canonisée (unique, triée)
        selected_canon[lbl] = sorted({normalize_label(x) for x in checked if normalize_label(x)})

    globals()['selected_by_config_raw'] = selected_raw
    globals()['selected_by_config_canonical'] = selected_canon

    with save_out:
        clear_output()
        print("✅ Sélections enregistrées dans :")
        print("   - selected_by_config_raw")
        # petit résumé
        for k in selected_raw:
            print(f"\t• {k}: {len(selected_raw[k])} canaux choisis => {selected_raw[k]}")

btn_save.on_click(collect_selection)
display(widgets.HBox([btn_save]), save_out)

<hr style="height:4px; background-color:black; border:none;">

## 3. Rename channels

The code cell below will open a widget where you can define a new label for the previously selected channels per configuration.
- Click a configuration to unfold its channels list.
- Use the textboxes to define new labels.
- Suggestions following the 10-10 international system can already be proposed.
- Click the "(Re)apply rules to all" to retrieve the suggestions.
- The new labels can be different from the 10-10 system but it can be a problem to plot topomaps in your future analyses.
- Repeat this operation for each configuration.
- Click the "Save remapping" button to save your inputs, a short summary will be displayed.\
Make sure to click on the "Save remapping" button before moving on to the next step.

In [ ]:
# -----------------------------
# 1) Normalization and synonyms
# -----------------------------

# 10–20 core (with official mixed case)
COMMON_10_20 = [
    "Fp1","Fp2","F7","F3","Fz","F4","F8",
    "T3","C3","Cz","C4","T4",
    "T5","P3","Pz","P4","T6",
    "O1","O2","T7","T8","P7","P8",
    "M1","M2","EOG_L","EOG_R"
]

# 10–10 extended (official mixed case; includes z in lowercase, Fp with p lowercase, etc.)
COMMON_10_10 = [
    # Frontal pole
    "Fp1", "Fpz", "Fp2",
    # Frontal
    "AF7", "AF3", "AFz", "AF4", "AF8",
    "F7", "F5", "F3", "F1", "Fz", "F2", "F4", "F6", "F8",
    # Frontocentral
    "FT7", "FC5", "FC3", "FC1", "FCz", "FC2", "FC4", "FC6", "FT8",
    # Central
    "T7", "C5", "C3", "C1", "Cz", "C2", "C4", "C6", "T8",
    # Centroparietal
    "TP7", "CP5", "CP3", "CP1", "CPz", "CP2", "CP4", "CP6", "TP8",
    # Parietal
    "P7", "P5", "P3", "P1", "Pz", "P2", "P4", "P6", "P8",
    # Parieto-occipital
    "PO7", "PO5", "PO3", "POz", "PO4", "PO6", "PO8",
    # Occipital
    "O1", "Oz", "O2",
    # Mastoid
    "M1", "M2",
    # EOG (not strictly 10–10 but commonly used)
    "EOG_L", "EOG_R",
]

# Build a case map: normalized key -> official mixed-case label
def _keyize(s: str) -> str:
    """Uppercase and strip non-alphanumerics for matching."""
    return re.sub(r"[^A-Z0-9]", "", str(s).strip().upper())

CASE_MAP = { _keyize(lbl): lbl for lbl in COMMON_10_10 }

# Synonyms (match by uppercase key; values should be final targets you want)
# Example: FZREF -> Fz, CZREF -> Cz, A1->M1, LOC->EOG_L, etc.
SYNONYMS_RAW = {
    # Eyes / EOG
    "LOC": "EOG_L", "ROC": "EOG_R",
    "E1": "EOG_L", "E2": "EOG_R",
    "EOGLEFT": "EOG_L", "EOGRIGHT": "EOG_R",
    # Mastoids / alternates
    "A1": "M1", "A2": "M2",
    # Explicit REF variants (anywhere they appear intact)
    "FZREF": "Fz", "CZREF": "Cz", "PZREF": "Pz",
}
SYNONYMS = { _keyize(k): v for k, v in SYNONYMS_RAW.items() }

def normalize_label(raw: str) -> str:
    """
    Return canonical EEG label using 10–10 official casing.
    Steps:
      0) strip common modality prefixes (EEG, EOG, EMG, ECG, EKG) preserving the remainder
      1) clean & uppercase for matching
      2) apply synonyms (case-insensitive)
      3) strip trailing REF/M1/M2/A1/A2
      4) re-case using CASE_MAP (10–10)
      5) apply cautious heuristics ONLY for known 10–10 families
      6) if still unknown -> return original label (stripped), not Title-case
    """
    if raw is None:
        return ""

    # 0) Retirer les préfixes de modalité : "EEG F3" -> "F3"
    # (on garde l'original pour le fallback final)
    original = str(raw).strip()
    pre = re.sub(r'^(EEG|EOG|EMG|ECG|EKG)[\s_-]+', '', original, flags=re.IGNORECASE)

    # 1) Keyize
    s_clean = _keyize(pre)  # uppercase + strip separators

    # 2) Synonymes
    if s_clean in SYNONYMS:
        target = SYNONYMS[s_clean]
        key_t = _keyize(target)
        return CASE_MAP.get(key_t, target)

    # 3) Retirer suffixes de référence
    s_clean = re.sub(r"(M1|M2|A1|A2|REF)$", "", s_clean)

    # 4) Mapping direct 10–10
    if s_clean in CASE_MAP:
        return CASE_MAP[s_clean]

    # 5) Heuristiques PRUDENTES, seulement si le préfixe fait partie d'une famille 10–10
    #    (évite de bricoler des labels comme "EEGF3")
    KNOWN_FAMILIES = {
        "FP","AF","F","FT","FC","C","CP","TP","P","PO","O","T","M","EOG"
    }
    m = re.match(r"([A-Z]+)(\d*Z?)$", s_clean)  # capture lettres + chiffres (et z éventuel)
    if m:
        letters, digits = m.groups()
        # autoriser heuristiques si le préfixe appartient à une famille connue
        # cas particulier: "FZ", "CZ", etc. -> gérer le 'z' minuscule
        if any(letters.startswith(fam) for fam in KNOWN_FAMILIES):
            # z final en minuscule pour les montages '...Z'
            if letters.endswith("Z") and len(letters) >= 2:
                return letters[:-1].title() + "z"
            # FP -> Fp + digits (inclut Fpz si digits=="Z")
            if letters.startswith("FP"):
                if digits.upper() == "Z":
                    return "Fpz"
                return "Fp" + digits.lower()
            # fallback léger: Title-case des lettres pour familles connues
            return letters.title() + digits

    # 6) Si on n'a rien reconnu ou heuristiques non applicables -> rendre l'ORIGINAL
    return original
    
# ----------------------------------------------------------------
# 2) Build the suggestion pool = official 10–10 + normalized from selections
# ----------------------------------------------------------------
if 'selected_by_config_raw' not in globals():
    raise RuntimeError("selected_by_config_raw not found. Run the selection widget first.")

config_labels = list(selected_by_config_raw.keys())

suggest_pool = set(COMMON_10_10)  # start with the official 10–10 mixed-case labels
for cfg_label in config_labels:
    for raw in selected_by_config_raw[cfg_label]:
        if raw:
            suggest_pool.add(normalize_label(raw))

SUGGESTIONS = sorted(x for x in suggest_pool if x)
# -----------------------------------------------------------
# 3) Editor: one Accordion tab per config, rows with Combobox
# -----------------------------------------------------------
row_widgets_by_cfg = {}  # {cfg_label: {raw_label: Combobox}}

def make_row(raw_label: str):
    """Return (HBox, Combobox) for raw -> canonical mapping."""
    # Combobox = suggestions + free text
    combo = widgets.Combobox(
        options=SUGGESTIONS,
        value=normalize_label(raw_label),        # pre-fill with a suggestion
        placeholder="Type or pick a canonical label…",
        ensure_option=False,                     # allow values outside the options list
        description="",                          # no left description (we show raw label separately)
        layout=widgets.Layout(width="240px")
    )
    raw_lab = widgets.Label(raw_label, layout=widgets.Layout(width="220px"))
    row = widgets.HBox([raw_lab, combo])
    return row, combo

panels = []
for cfg_label in config_labels:
    row_widgets_by_cfg[cfg_label] = {}

    # stable ordering
    raw_list = sorted(selected_by_config_raw[cfg_label], key=lambda s: s.upper())

    # Local toolbar
    btn_apply_rules = widgets.Button(
        description="(Re)apply rules to all",
        tooltip="Re-run normalize_label(raw) for every row in this configuration",
        button_style="info"
    )
    info = widgets.HTML(value="<i>You can type freely or pick a suggestion.</i>")

    # Rows
    rows = []
    for raw in raw_list:
        row, combo = make_row(raw)
        rows.append(row)
        row_widgets_by_cfg[cfg_label][raw] = combo

    # Bind apply-all
    def make_apply_all(rows_map=row_widgets_by_cfg[cfg_label], raws=raw_list):
        def fn(_):
            for r in raws:
                rows_map[r].value = normalize_label(r)
        return fn
    btn_apply_rules.on_click(make_apply_all())

    panel = widgets.VBox([
        widgets.HBox([btn_apply_rules, info]),
        widgets.VBox(
            rows,
            layout=widgets.Layout(max_height="380px", overflow="auto", border="1px solid #ddd", padding="6px")
        )
    ])

    panels.append(panel)

acc = widgets.Accordion(children=panels)
for i, cfg_label in enumerate(config_labels):
    acc.set_title(i, cfg_label)

display(acc)

# === 10–10 helpers ===
ALLOWED_NON_TENTEN = {"EOG_L", "EOG_R"}  # tolérés au même titre que 10–10

def is_ten_ten(label: str) -> bool:
    """True si le label est dans la nomenclature 10–10 (CASE_MAP) ou explicitement autorisé."""
    if not label:
        return False
    key = _keyize(label)  # même keyizer que pour CASE_MAP
    return (key in CASE_MAP) or (label in ALLOWED_NON_TENTEN)

# --------------------------------------------
# 4) Save mapping -> remap_by_config + warnings + exports
# --------------------------------------------

btn_save = widgets.Button(description="Save remapping", button_style="success", icon="save")
out = widgets.Output(
    layout=widgets.Layout(
        width="100%",
        height="auto",
        max_height="none",
        overflow="visible",      # <- enlève la scrollbox
        border="0"  # optionnel
    )
)
display(widgets.HBox([btn_save]), out)

def on_save(_=None):
    export_msg = ""
    warnings_dup = []      # [(cfg_label, {canon: [raws...]})]
    nonstandard = {}       # {cfg_label: [non-10–10 labels]}
    try:
        # --- build mappings ---
        remap = {}
        canonical_lists = {}

        for cfg_label in config_labels:
            rows_map = row_widgets_by_cfg[cfg_label]   # {raw: Combobox}
            mapping = {}
            bad = []
            for raw, combo in rows_map.items():
                can = (combo.value or "").strip()
                mapping[raw] = can
                # vérif stricte 10–10 (PAS de garde)
                if can and not is_ten_ten(can):
                    bad.append(can)

            remap[cfg_label] = mapping
            canonical_lists[cfg_label] = sorted({v for v in mapping.values() if v})
            if bad:
                nonstandard[cfg_label] = sorted(set(bad))

        # doublons de cibles canonisées (info)
        for cfg_label, mapping in remap.items():
            inv = {}
            for raw, can in mapping.items():
                if not can:
                    continue
                inv.setdefault(can, []).append(raw)
            dups = {k: v for k, v in inv.items() if len(v) > 1}
            if dups:
                warnings_dup.append((cfg_label, dups))

        # exposer en globals
        globals()["remap_by_config"] = remap
        globals()["selected_by_config_canonical"] = canonical_lists

        # --- exports ---
        try:
            # mapping complet
            df_rows = []
            for cfg_label, mapping in remap.items():
                for raw, can in mapping.items():
                    df_rows.append({"config": cfg_label, "raw_channel": raw, "canonical_channel": can})
            df_map = pd.DataFrame(df_rows)
            # commenting the save for now
            # df_map.to_csv(f"{config_param_path}/remap_raw_to_canonical.tsv", sep="\t", index=False)

            # listes canoniques par config
            df_canon = pd.DataFrame({cfg: pd.Series(chs) for cfg, chs in canonical_lists.items()})
            # commenting the save for now
            # df_canon.to_csv(f"{config_param_path}/selected_canonical_by_config.tsv", sep="\t", index=False)

            # labels non 10–10
            if nonstandard:
                ns_rows = []
                for cfg_label, labs in nonstandard.items():
                    for lab in labs:
                        ns_rows.append({"config": cfg_label, "non_10_10_label": lab})
                # commenting the save for now
                # pd.DataFrame(ns_rows).to_csv(f"{config_param_path}/non_10_10_labels.tsv", sep="\t", index=False)

            export_msg = (f"\n📝 Exports:\n"
                          f" - {config_param_path}/remap_raw_to_canonical.tsv\n"
                          f" - {config_param_path}/selected_canonical_by_config.tsv"
                          + (f"\n - {config_param_path}/non_10_10_labels.tsv" if nonstandard else ""))
        except Exception as e:
            export_msg = f"\n(Export skipped: {e})"

        # Export JSON
        try:
            # commenting the save for now (20/11/2025)
            json_path = os.path.join(config_param_path, "mne_remap_plan.json")
            # with open(json_path, "w", encoding="utf-8") as f:
            #     json.dump(remap_by_config, f, indent=2, ensure_ascii=False)
            saved_msg = f"    JSON:\n - {json_path}"
        except Exception as e:
            saved_msg = f"(JSON export skipped: {e})"
        print(saved_msg)

        # --- affichage ---
        with out:
            clear_output()
            print("✅ Mapping saved to variables:")
            print("   - remap_by_config")
            print("   - selected_by_config_canonical")
            # commenting the export because they are saved later per participant
            # print(export_msg)
            # print(saved_msg)

            if warnings_dup:
                print("\n⚠️ Multiple raw labels mapped to the same canonical within a configuration:")
                for cfg_label, dups in warnings_dup:
                    print(f"  • {cfg_label}: {dups}")

            if nonstandard:
                print("\n⚠️ Non-10–10 canonical labels detected (consider official 10–10 names):")
                for cfg_label, badlist in nonstandard.items():
                    print(f"  • {cfg_label}: {badlist}")
                print("⚠️ This might cause issue to plot topomap (this is not a problem for clinical set-up with 3 channels only)")

            if not warnings_dup and not nonstandard:
                print("\nNo issues detected.")

            print("\nQuick overview of the remapping:")
            for config, remap_d in remap_by_config.items():
                print(f"\t{config}: {remap_d}")

    except Exception:
        with out:
            clear_output()
            print("❌ Error while saving the mapping — traceback:")
            traceback.print_exc()

btn_save.on_click(on_save)

<hr style="height:4px; background-color:black; border:none;">

## 4. Define re-reference method

The code cell below will open a widget where you can define a re-reference method (to apply before your analysis).\
There is 3 options: 
- None (if your data are already re-referenced (e.g. C4-M1 is already referenced to M1)
- Average
- Custom (to choose one or two specific channels)
  
Steps of the widget:
- Click a configuration to unfold its re-reference selection.
- Choose your method of re-referencing.
- For custom, select the wanted reference channel and click the "Add" button.
- To use two channels as reference (e.g. linked mastoids M1-M2), you need to add each channel one by one (with the same instruction as above).
- Repeat this operation for each configuration.
- Click the "Save re-reference" button to save your inputs, a short summary will be displayed.\
Make sure to click on the "Save re-reference" button before moving on to the next step.

In [ ]:
# Inputs expected
if 'selected_by_config_canonical' not in globals():
    raise RuntimeError("selected_by_config_canonical not found. Run the previous mapping step first.")
if 'COMMON_10_10' not in globals():
    COMMON_10_10 = []  # optional

config_labels = list(selected_by_config_canonical.keys())

# Suggestions: union of 10–10 and each config channels (per config we’ll filter)
base_suggestions = set(COMMON_10_10)

# State storage
state_by_cfg = {}  # cfg -> dict(mode_radio, combo, add_btn, list_box, info_html)

def build_panel_for_config(cfg_label):
    """
    Left: configuration channels (read-only).
    Right: mode (None/Average/Custom) and a Combobox-based multi-pick for Custom.
    """
    cfg_channels = sorted(selected_by_config_canonical[cfg_label])
    # Suggestions for this config = its channels + 10-10
    suggestions = cfg_channels # use sorted(base_suggestions.union(cfg_channels)) if you want to add the 10-10 list

    # --- Left: show configuration channels ---
    left_title = widgets.HTML(f"<b>Configuration channels ({len(cfg_channels)}):</b>")
    left_list  = widgets.VBox(
        [widgets.HTML(", ".join(cfg_channels))],
        layout=widgets.Layout(max_height="150px", overflow="auto", border="1px solid #ddd", padding="6px")
    )
    left_box = widgets.VBox([left_title, left_list], layout=widgets.Layout(width="50%"))

    # --- Right: controls ---
    mode = widgets.RadioButtons(
        options=[("None (keep as-is)", "none"),
                 ("Average reference", "average"),
                 ("Custom reference (pick)", "custom")],
        value="none",
        description="Method:",
        layout=widgets.Layout(width="330px")
    )

    # Combobox to add ONE ref channel at a time (free text + suggestions)
    combo = widgets.Combobox(
        options=suggestions,
        value="",
        placeholder="Type or pick a reference channel…",
        ensure_option=False,   # allow values outside suggestions
        layout=widgets.Layout(width="260px")
    )
    add_btn = widgets.Button(description="Add", button_style="primary", tooltip="Add channel to custom reference list")

    # A list of currently chosen reference channels (with removable buttons)
    chosen_box = widgets.VBox([], layout=widgets.Layout(
        max_height="200px", overflow="auto", border="1px solid #ddd", padding="6px", width="260px"
    ))
    chosen_label = widgets.HTML("<b>Custom reference channels:</b>")

    # Info / validation
    info = widgets.HTML("<i>Select re-reference method. For 'Custom', add channels using the combobox.</i>")

    # Helper to (re)build the chosen list UI
    chosen = []  # Python list of strings (unique)
    def refresh_chosen_box():
        # Clear and re-create rows with a small remove (×) button
        rows = []
        for ch in chosen:
            rm_btn = widgets.Button(description="×", tooltip=f"Remove {ch}", layout=widgets.Layout(width="28px"))
            lbl = widgets.Label(ch)
            def make_rm(target=ch):
                def _(_b):
                    if target in chosen:
                        chosen.remove(target)
                        refresh_chosen_box()
                return _
            rm_btn.on_click(make_rm())
            rows.append(widgets.HBox([rm_btn, lbl]))
        chosen_box.children = rows

    # Add channel from combobox
    def on_add(_):
        ch = (combo.value or "").strip()
        if not ch:
            return
        # Deduplicate
        if ch not in chosen:
            chosen.append(ch)
            chosen.sort()
            refresh_chosen_box()
        combo.value = ""  # clear input for next entry

    add_btn.on_click(on_add)

    # Enable/disable custom area by mode
    def set_custom_enabled(enabled: bool):
        combo.disabled = not enabled
        add_btn.disabled = not enabled
        # You can still view/remove chosen even if disabled; leave chosen_box enabled.

    def on_mode_change(change):
        if change["name"] == "value":
            m = change["new"]
            if m == "none":
                info.value = "<i>No re-referencing will be applied for this configuration.</i>"
                set_custom_enabled(False)
            elif m == "average":
                info.value = "<i>MNE: <code>raw.set_eeg_reference('average')</code>.</i>"
                set_custom_enabled(False)
            else:
                info.value = "<i>Pick one or more channels to use as reference (MNE: <code>raw.set_eeg_reference(ref_channels=[...])</code>).</i>"
                set_custom_enabled(True)

    mode.observe(on_mode_change, names="value")
    set_custom_enabled(False)  # start in "none"

    right_top = widgets.VBox([mode, info])
    right_custom = widgets.VBox([
        widgets.HBox([combo, add_btn]),
        chosen_label,
        chosen_box
    ])
    right_box = widgets.VBox([right_top, right_custom], layout=widgets.Layout(width="50%"))

    # Store state (chosen list lives in closure but also store handle for save)
    state_by_cfg[cfg_label] = {
        "mode": mode,
        "combo": combo,
        "add_btn": add_btn,
        "chosen_list_ref": chosen,   # the Python list to read at save time
        "config_channels": cfg_channels,
        "info": info
    }

    return widgets.HBox([left_box, right_box], layout=widgets.Layout(gap="16px", align_items="flex-start"))

# Build accordion
panels = [build_panel_for_config(cfg) for cfg in config_labels]
acc_reref = widgets.Accordion(children=panels)
for i, cfg in enumerate(config_labels):
    acc_reref.set_title(i, f"{cfg} — re-reference")
display(acc_reref)

# Save button
btn_save_plan = widgets.Button(description="Save re-reference", button_style="success", icon="save", layout=widgets.Layout(width='150px'))
out_plan = widgets.Output()
display(widgets.HBox([btn_save_plan]), out_plan)

def validate_choice(cfg_label, mode, ref_chans, cfg_channels):
    """Return (ok, message). Warn if custom refs not present in this config."""
    if mode == "custom":
        if len(ref_chans) == 0:
            return False, "Please add at least one reference channel for 'Custom' mode."
        missing = [ch for ch in ref_chans if ch not in cfg_channels]
        if missing:
            return False, f"Some chosen reference channels are not in this configuration: {missing}"
    return True, mode

def on_save_plan(_=None):
    """
    Build a dict ready for MNE re-referencing, per configuration:
      - 'none'
      - 'average'
      - 'custom' + ref_channels: [...]
    Export JSON for later reuse.
    """
    plan = {}
    messages = []
    ok_all = True

    for cfg in config_labels:
        st = state_by_cfg[cfg]
        mode = st["mode"].value
        refs = list(st["chosen_list_ref"])  # copy
        cfg_chs = st["config_channels"]

        ok, msg = validate_choice(cfg, mode, refs, cfg_chs)
        if not ok:
            ok_all = False
        messages.append(f"{cfg}: {msg}")

        if mode == "none":
            spec = {"ref_channels": []}
        elif mode == "average":
            spec = {"ref_channels": "average"}
        else:
            spec = {"ref_channels": refs}

        plan[cfg] = spec

    globals()["reref_plan_by_config"] = plan

    # Export JSON
    try:
        out_json = os.path.join(config_param_path, "mne_reref_plan.json")
        with open(out_json, "w", encoding="utf-8") as f:
            json.dump(plan, f, indent=2, ensure_ascii=False)
        saved_msg = f"Saved JSON:\n  - {out_json}"
    except Exception as e:
        saved_msg = f"(JSON export skipped: {e})"

    with out_plan:
        clear_output()
        print("✅ Re-reference plan saved to variable: reref_plan_by_config")
        print(saved_msg)
        print("\nSummary:")
        for m in messages:
            print(" - " + m)
        if not ok_all:
            print("\n⚠️ Please fix the warnings above before applying to MNE.")

btn_save_plan.on_click(on_save_plan)

<hr style="height:4px; background-color:black; border:none;">

## 5. Preview and save JSON file

The code cell below will open a widget to preview the resulting file and save it.
- Name the file as you please.
- Click on "Preview" to display the first rows of the file. You can inspect if it corresponds to your expectations. 
- Click on the "Save" button to save the .json file.

In [ ]:
# === Widget minimaliste : aperçu JSON exact + sauvegarde forcée ===
# Prérequis dans l'environnement :
# - ch_config_dict : { tuple(sorted(set(raw_channels))) : [list_of_subjects] }
# - configs        : list(tuple(...))          # même ordre que config_labels
# - config_labels  : list[str]                  # ex. "config. 1 (n=26)"
# - remap_by_config: { "config. i (n=…)" : {raw_channel: canonical_channel, ...} }
# - (optionnel) reref_plan_by_config : { "config. i (n=…)" : {"ref_channels": ...} }
# - config_param_path : dossier où sauvegarder le JSON

# ---------- logique de construction ----------
def _as_label_per_config(configs, config_labels):
    if len(configs) != len(config_labels):
        raise RuntimeError(
            "Inconsistency: 'configs' and 'config_labels' have different lengths."
        )
    return {tuple(cfg): lab for cfg, lab in zip(configs, config_labels)}

def _subject_to_config_label(ch_config_dict, cfg2label):
    out = {}
    for cfg_tuple, subs in ch_config_dict.items():
        lab = cfg2label.get(tuple(cfg_tuple))
        if lab is None:
            # tolérance à l’ordre: on tente la correspondance par set-égalité
            set_cfg = set(cfg_tuple)
            for k_tuple, k_lab in cfg2label.items():
                if set(k_tuple) == set_cfg:
                    lab = k_lab
                    break
            if lab is None:
                raise KeyError(f"Config tuple not found in the mapping configs→labels:\n{cfg_tuple}")
        for s in subs:
            out[s] = lab
    return out

def _normalize_ref_value(ref_value):
    if isinstance(ref_value, list):
        if len(ref_value) == 0:
            return []
        if len(ref_value) == 1:
            return [ref_value[0]]
        return ref_value
    return ref_value  # 'average' ou autre chaîne

def build_per_subject_dict():
    needed = ["ch_config_dict", "configs", "config_labels", "remap_by_config", "config_param_path"]
    missing = [v for v in needed if v not in globals()]
    if missing:
        raise RuntimeError(f"Missing(s) variable(s): {', '.join(missing)}")
    if not os.path.isdir(config_param_path):
        raise RuntimeError(f"'config_param_path' is not a valid folder: {config_param_path}")

    cfg2label = _as_label_per_config(configs, config_labels)
    subject_to_cfg_label = _subject_to_config_label(ch_config_dict, cfg2label)

    per_subject = OrderedDict()
    for sub, cfg_label in sorted(subject_to_cfg_label.items(), key=lambda kv: kv[0]):
        remap_map = remap_by_config.get(cfg_label, {})
        if "reref_plan_by_config" in globals():
            ref_spec = reref_plan_by_config.get(cfg_label, {"ref_channels": []})
            ref_val = _normalize_ref_value(ref_spec.get("ref_channels", []))
        else:
            ref_val = None

        # Compacte toujours le label de config
        cfg_compact = re.sub(r"\s*\(n=\d+\)\s*$", "", cfg_label)

        per_subject[sub] = {
            "config": cfg_compact,
            "remap": remap_map,
            "ref_channels": ref_val
        }
    return per_subject

# ---------- UI ----------
title = widgets.HTML("<h3>Participant → {config, remap, ref_channels}</h3>")
fname_text = widgets.Text(value="remap_reref_persubject.json", description="Fichier:", layout=widgets.Layout(width="420px"))

btn_preview = widgets.Button(description="Preview", button_style="info", icon="eye")
btn_save = widgets.Button(description="Save", button_style="success", icon="save", disabled=True)

preview_out = widgets.Output(layout=widgets.Layout(border="1px solid #444", padding="6px", max_height="360px", overflow="auto"))
status_out = widgets.Output()

_state = {"dict": None, "path": None, "json_text": None}

def _render_json_block(json_text, max_lines=41):
    """
    Affiche seulement les `max_lines` premières lignes du JSON
    pour éviter d'alourdir le widget quand il est très gros.
    """
    lines = json_text.splitlines()
    if len(lines) > max_lines:
        preview = "\n".join(lines[:max_lines]) + f"\n... ({len(lines)-max_lines} lignes supplémentaires masquées)"
    else:
        preview = json_text
    style = "margin:0; white-space:pre; font-family:Menlo,Consolas,monospace; font-size:12px;"
    return f"<pre style='{style}'>{preview}</pre>"

def _make_preview():
    d = build_per_subject_dict()
    out_json = os.path.join(config_param_path, fname_text.value.strip() or "remap_reref_persubject.json")
    json_text = json.dumps(d, indent=2, ensure_ascii=False)
    _state.update(dict(dict=d, path=out_json, json_text=json_text))

@btn_preview.on_click
def _on_preview(_):
    preview_out.clear_output()
    status_out.clear_output()
    try:
        _make_preview()
        with preview_out:
            display(HTML(f"<p style='margin:0 0 6px 0'><b>JSON will be saved here :</b> <code>{_state['path']}</code></p>"))
            display(HTML(_render_json_block(_state["json_text"])))
        btn_save.disabled = False
    except Exception as e:
        btn_save.disabled = True
        with preview_out:
            display(HTML(f"<pre style='color:#c33'>{type(e).__name__}: {e}</pre>"))

@btn_save.on_click
def _on_save(_):
    status_out.clear_output()
    if not _state.get("dict") or not _state.get("path"):
        with status_out:
            display(HTML("<span style='color:#c33'>Click first on <b>Preview</b>.</span>"))
        return
    out_json = _state["path"]
    STATE.json_path = out_json
    try:
        os.makedirs(os.path.dirname(out_json), exist_ok=True)
        with open(out_json, "w", encoding="utf-8") as f:
            f.write(_state["json_text"])
        with status_out:
            display(HTML(f"<span style='color:#2b8a3e'>✅ JSON saved (overwrite) : <code>{out_json}</code></span>"))
            display(HTML(f"<br>To load your file, use these coding lines:"))
            display(HTML(f"<p>- <code>json_path = {out_json}</code></p>"))
            display(HTML(f"<p>- <code>with open(json_path, 'r', encoding='utf-8') as f: config_dict = json.load(f)</code></p>"))
            display(HTML(f"<br>To select the info of a participant:"))
            display(HTML(f"<p>- <code>file_ID = os.path.basename(edf_path).split('.')[0]</code></p>"))
            display(HTML(f"<p>- <code>sub_config = config_dict[file_ID]</code></p>"))
            display(HTML(f"<br>To load the data with only the selected channels:"))
            display(HTML(f"<p>- <code>raw = mne.io.read_raw_edf(edf_path, preload=True, include=list(sub_config['remap'].keys()))</code></p>"))
            display(HTML(f"<br>To rename the selected channels:"))
            display(HTML(f"<p>- <code>raw.rename_channels(sub_config['remap'])</code></p>"))
            display(HTML(f"<br>To apply the re-reference method:"))
            display(HTML(f"<p>- <code>raw.set_eeg_reference(ref_channels = sub_config['ref_channels'])</code></p>"))
            display(HTML(f"<p>- <code>raw.drop_channels(sub_config['ref_channels'])</code></p>"))

        globals()["per_subject_dict"] = _state["dict"]
    except Exception as e:
        with status_out:
            display(HTML(f"<pre style='color:#c33'>{type(e).__name__}: {e}</pre>"))

# ---------- agencement ----------
controls = widgets.HBox([fname_text, btn_preview, btn_save])
box = widgets.VBox([title, controls, preview_out, status_out])
display(box)

# Pour générer automatiquement l'aperçu à l'exécution :
# _on_preview(None)

<hr style="height:4px; background-color:black; border:none;">

## 6. Test the JSON file

The code cell below will loop across your database and apply the remapping and re-reference.
It will return errors if it does not succeed on some files.

In [ ]:
# check the remapping for the full dataset

# define custom functions to take into account duplicate channels that will be labelled XX-0 and XX-1 by mne
# we keep only the XX-0 and then re-define a dict to select the XX-0 instead of XX
# this is necessary for APOMORPHE database
def drop_suffix_duplicates(raw):
    """
    Repère les channels de type 'XXX-0', 'XXX-1', ...
    et ne garde que les '-0' lorsqu'ils coexistent.
    
    Exemple :
    - ['C4-0', 'C4-1', 'F4', 'O2'] -> on drop 'C4-1'
    - ['C4-1'] tout seul -> on ne drop rien.
    """
    ch_names = raw.ch_names
    # regroupement par 'base', ex: C4-0 -> base = C4
    groups = {}
    for ch in ch_names:
        if ch.endswith('-0') or ch.endswith('-1'):
            base = ch.rsplit('-', 1)[0]
            groups.setdefault(base, []).append(ch)

    to_drop = []
    for base, ch_list in groups.items():
        zeros = [c for c in ch_list if c.endswith('-0')]
        if zeros:
            # si un '-0' existe, on drop tous les autres suffixés (typiquement les '-1')
            for c in ch_list:
                if not c.endswith('-0'):
                    to_drop.append(c)

    if to_drop:
        print("🔎 Channels suffixés trouvés, on va drop : ", to_drop)
        raw.drop_channels(to_drop)
    else:
        print("✅ Aucun doublon '-0'/'-1' à nettoyer")

    return raw, to_drop

def adapt_remap_dict_to_suffixes(raw, remap_dict):
    """
    Adapte un remap_dict de la forme {"C4": "C", "F4": "F", ...}
    aux noms de channels réellement présents dans raw.ch_names,
    en remplaçant par exemple "C4" -> "C4-0" si "C4" n'existe pas
    mais "C4-0" oui.

    Retourne un nouveau dict prêt pour raw.rename_channels().
    """
    ch_set = set(raw.ch_names)
    new_remap = {}

    for base_label, target in remap_dict.items():
        if base_label in ch_set:
            # Cas simple : le label existe tel quel dans raw.ch_names
            new_remap[base_label] = target
        else:
            # Cas suffixé : essayer base_label-0
            candidate = f"{base_label}-0"
            if candidate in ch_set:
                new_remap[candidate] = target
            # sinon : on ignore ce label (aucun ch correspondant)

    return new_remap
# load the output (dictionary) of the notebook
# json_path = _state['path']  # ton chemin complet
json_path = STATE.json_path  # ton chemin complet

with open(json_path, "r", encoding="utf-8") as f:
    config_dict = json.load(f)

# Initialize an empty list for files that could not be read
failed_file_list = []

# Initialize empty list for channels
remap_channels_list = []
raw_channels_list = []
check_ref_before = []
check_ref_after = []

# initialize a dynamic output
output_remap = ""
dynamic_remap_out = widgets.Output()
display(dynamic_remap_out)
# define a function to update the dynamic output
def update_scrollbox():
    """Rafraîchit la scrollbox avec le contenu de output_remap."""
    with dynamic_remap_out:
        dynamic_remap_out.clear_output(wait=True)
        print_in_scrollable_box(output_remap, font_size="12px")

# init mne_output
loop_out = widgets.Output()        # sorties MNE temporaires
display(loop_out)

with loop_out:
    # for e, edf_path in enumerate(edf_files):
    for e, edf_path in enumerate(SATE.edf_files):
        loop_out.clear_output(wait=True)
        # output_remap += (f'file {e+1}/{len(edf_files)}, currently remapping: {edf_path}\n')
        output_remap += (f'file {e+1}/{len(STATE.edf_files)}, currently remapping: {edf_path}\n')
        update_scrollbox()
            
        # read file with the custom function
        try:
            # extract file ID to get info from the dict
            # file_name = os.path.basename(raw_fn)  # Ex: "15_N1.edf"
            file_ID = os.path.basename(edf_path).split('.')[0]  # Extrait "15_N1"

            try:
                sub_config = config_dict[file_ID]
            except KeyError:
                err = f"\t❌ File ID {file_ID} was not found in the remapping dictionary\n"
                output_remap += err
                update_scrollbox()
                continue
            except Exception as e:
                err = f"\t❌ Unexpected problem with the remapping dictionary and file ID {file_ID}: {e}\n"
                output_remap += err
                update_scrollbox()
                continue

            selected_channels = list(sub_config["remap"].keys())

            # load edf as mne raw object
            try:
                raw = mne.io.read_raw_edf(edf_path, preload=True, include=selected_channels) # we need to preload to use  re-ref; old param from Thomas encoding="latin-1", 
            except Exception:
                err = f"\t❌ Unexpected problem in loading edf file with mne: {e}\n"
                output_remap += err
                update_scrollbox()
                continue

            # save original raw channels
            raw_channels_list.append(raw.ch_names)
            check_ref_before.append(raw.info['custom_ref_applied'])

            # remove duplicates channels
            raw, dropped = drop_suffix_duplicates(raw)

            # remap dict to take into account XX-0 created by mne for duplicates
            remap_dict_adapted = adapt_remap_dict_to_suffixes(raw, sub_config["remap"])

            # remap channels (based on the generated dict from jupy notebook):
            raw.rename_channels(remap_dict_adapted)
            
            # set eeg reference
            if  sub_config["ref_channels"] != 'average':
                raw.set_eeg_reference(ref_channels = sub_config["ref_channels"])
                # get rid of the ref
                raw.drop_channels(sub_config["ref_channels"])
            else:
                raw.set_eeg_reference(ref_channels = sub_config["ref_channels"])

            remap_channels_list.append(raw.ch_names)
            check_ref_after.append(raw.info['custom_ref_applied'])


            del raw

        except Exception as e:
            err = f"\t❌ Unexpected problem for {edf_path} : {e}\n"
            output_remap += err
            update_scrollbox()
            failed_file_list.append((edf_path, 'other'))
    loop_out.clear_output()

# check the remapping
unique_ch = list({tuple(sorted(x)) for x in raw_channels_list})
unique_remap = list({tuple(sorted(x)) for x in remap_channels_list})
print(f"Possible channels configurations before remapping: {len(unique_ch)}") # can be different than the number of configurations earlier because it is for the selected channels 
for c, chs in enumerate(unique_ch):
    print(f"\t{chs}")
print(f"\nPossible channels configurations after remapping: {len(unique_remap)}")
for c, chs in enumerate(unique_remap):
    print(f"\t{chs}")

if len(unique_remap) == 1:
    print(f"\n✔️ The remapping worked ! You can safely use the remapping dictionary for your analyses.")
elif len(unique_remap) > 1:
    print(f"\n❌ The remapping failed. There is still multiple channels configurations.")
elif len(unique_remap) < 1:
    print(f"\n❌ The remapping failed. There is no channels configuration left.")
else:
    print(f"\n❌ The remapping failed. Unknown error.")